In [7]:
import torch

torch.randn(10, 2)[:-1].shape

torch.Size([9, 2])

In [1]:
from modeling import InterpretableMultiHeadAttention, InterpretableMultiHeadAttention2

C:\Users\janba\python-dev\kernel_attention_TFT\.venv\Lib\site-packages\torch\_subclasses\functional_tensor.py:258: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [2]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [47]:
config = {
    "n_head": 4,
    "hidden_size": 128,
    "dropout": 0.1,
    "example_length": 192,
    "attn_dropout": 0.0,
    "batch_size": 1
}
config = dotdict(config)

In [48]:
import torch

x = torch.randn(config.batch_size, config.example_length, config.hidden_size)

In [75]:
import attention as ATTN

nH = config.n_head
t = config.example_length
nD = config.hidden_size // config.n_head

qk = torch.randn(64, t, nH, nD)
v = torch.randn(64, t, nD)

qk.shape, v.shape

(torch.Size([64, 192, 4, 32]), torch.Size([64, 192, 32]))

In [86]:
y, attn_w = ATTN.DotProductAttention()(qk.transpose(1, 2), qk.transpose(1, 2), v.unsqueeze(1))
y, attn_w = ATTN.ChangePointKernelAttention()(qk.transpose(1, 2), qk.transpose(1, 2), v.unsqueeze(1))

y.shape, attn_w.shape

(torch.Size([64, 4, 192, 32]), torch.Size([64, 4, 192, 192]))

In [51]:
imha = InterpretableMultiHeadAttention(config, ATTN.DotProductAttention(0.0))
imha2 = InterpretableMultiHeadAttention2(config)

imha.eval()
imha2.eval()

for name, p in imha.named_parameters():
    torch.nn.init.ones_(p)

for name, p in imha2.named_parameters():
    torch.nn.init.ones_(p)

In [74]:
with torch.no_grad():
    y, attn_w = imha(x)
y.shape, attn_w.shape

(torch.Size([1, 192, 128]), torch.Size([1, 4, 192, 192]))

In [73]:
with torch.no_grad():
    y2, attn_w2 = imha2(x)
y2.shape, attn_w2.shape

(torch.Size([1, 192, 128]), torch.Size([1, 4, 192, 192]))